# ADR (Anomaly Detection by workflow Relations)

ADR mines numerical relations from log data and uses the relations for anomaly detection.

In the following parts, we use the BGL logs as an example to show the capability of ADR.

## parse raw logs to log events and build the event count matrix

The example raw logs are in "_data/BGL_2k.log_".

For ease of presentation, the raw logs are already parsed into structured log events by Drain <sup>[1]</sup> and the parsed results are in "_data/Drain_result/bgl_" folder. The file "_BGL_2k.log_structured.csv_" are the parsed structured logs and the file "_BGL_2k.log_templates.csv_" are the templates (events) of the logs.

## Build the event count matrix

In [1]:
from ADR import preprocess
import pandas as pd

print("Loading...")
log_path = 'logs/HDFS.log_structured.csv'
label_path = 'logs/anomaly_label.csv'
template_path = 'logs/'

# df_log = pd.read_csv(log_path, sep=',', header=0)
# eventID_list = pd.read_csv(template_path, sep=',', header=0)['EventId'].tolist()
df_log = preprocess.load_hdfs_structured_logs(log_path, label_path)
print("Loaded!")
# df_log["bLabel"] = True
# df_log.loc[df_log["Label"]=="-", "bLabel"] = False

seq_df, seq_ecm_df = preprocess.event_sequence_by_identifier(df_log, col_identifier='BlockId', col_EventId='EventId',
                                                             col_bLabel='bLabel')

seq_ecm_df = seq_ecm_df.fillna(0).astype(int)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Loading...
extracting block_id...


Loaded!


  0%|          | 0/11175629 [00:00<?, ?it/s]


KeyboardInterrupt



In [18]:
# print the session sequences and the events of each session
seq_df

,bLabel,seq_EventId,seq_LineId,seq_bLabel
sadmin1,True,"[5947eee3, ce370d35, 77197260, 77197260, c6e07...","[1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 19, 20, 21,...","[False, False, True, True, True, True, True, T..."
sadmin2,False,"[87b68315, ff46cb1d, ff46cb1d, 5947eee3, ff46c...","[2, 3, 4, 32, 33, 34, 35, 36, 37, 577, 578, 57...","[False, False, False, False, False, False, Fal..."
sn209,False,"[38bd5405, 2a2db037, fadd4d28, 182a3d2f, 5947e...","[14, 15, 16, 17, 109, 561, 562, 563, 564, 596,...","[False, False, False, False, False, False, Fal..."
shpnfs,False,"[d7b7a5ec, 5947eee3, d7b7a5ec, d7b7a5ec, a7683...","[18, 38, 555, 626, 672, 724, 777, 856, 906, 94...","[False, False, False, False, False, False, Fal..."
sn504,False,"[182a3d2f, 5947eee3, 182a3d2f, 182a3d2f, 38bd5...","[24, 292, 551, 552, 569, 570, 571, 582, 586, 5...","[False, False, False, False, False, False, Fal..."
...,...,...,...,...
sn409,False,"[5947eee3, 5947eee3, 5947eee3, 5947eee3, 5947e...","[554, 2448, 5064, 6996, 10153, 11813, 12808, 1...","[False, False, False, False, False, False, Fal..."
sn471,False,"[8c537990, c508951e, e5584a32, 4124b970, 444bc...","[189734, 189735, 189736, 189737, 189738, 18973...","[False, False, False, False, False, False, Fal..."
sn250,True,"[6dc36b09, 0f5f997d, 0f5f997d, 6dc36b09, 5209b...","[263684, 263685, 263686, 263687, 263688, 26368...","[False, False, False, False, False, False, Fal..."
sn355,False,"[6dc36b09, 0f5f997d, 0f5f997d, 6dc36b09, 5209b...","[265506, 265507, 265508, 265509, 265510, 26551...","[False, False, False, False, False, False, Fal..."


In [19]:
# print the event count matrix
seq_ecm_df

,77197260,c6e07261,fc10e26c,ce370d35,9ce775c4,e8b0f746,2b97ca7e,dc59a28d,f117df68,c89a99ae,...,30664715,690d155d,d0ce5cf3,4fb89c05,8b176746,0703bd02,09e9d8dc,ce9fb8d3,ba3f7700,7b5753a9
sadmin1,200254,200254,96904,76999,41633,4230,4020,4020,4020,4020,...,0,0,0,0,0,0,0,0,0,0
sadmin2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sn209,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
shpnfs,0,0,0,0,249,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sn504,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sn409,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sn471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
sn250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sn355,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [20]:
x, y = [], []
for idx, r in seq_df.iterrows():
    y.append(r['bLabel'])
    x.append(seq_ecm_df.loc[idx, :].tolist())
y = [int(l) for l in y]

## load datasets

The example log with 2k lines are too small to be used for anomaly detection. So we use the event count matrix of the whole BGL dataset for the anomaly detection demo.

In [21]:
import numpy as np
import pickle

x = np.array(x, dtype=float)
y = np.array(y, dtype=int)

with open("spirit.pkl", mode="wb") as f:
    pickle.dump((x, y), f, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
import pickle

with open("spirit.pkl", mode="rb") as f:
    (x, y) = pickle.load(f)

## sADR (semi-supervised, need normal logs for training)

In [24]:
from ADR import preprocess
from ADR import sADR

res = []
for t in range(1):
    print(f'******* time:{t} ********')
    train_ratio = [400]
    r_res = []
    for r in train_ratio:
        # train_number = train_numbers[i]
        print(f'-----train ratio:{r}-----')
        # if i == 0:
        #     x_train, y_train, x_test, y_test = x_train, y_train, x_test, y_test = preprocess.split_to_train_test_by_num(x, y, num_train=train_number)
        # else:
        x_train, y_train, x_test, y_test = preprocess.split_to_train_test_by_num(x, y, num_train=r)
        # x_train = np.concatenate((x_train, x_train_adding), axis=0)
        # y_train = np.concatenate((y_train, y_train_adding), axis=0)

        # print(np.arange(x_train.shape[0]))
        model = sADR.sADR()
        model.fit(x_train, y_train, sample=False)
        # precision, recall, f1 = model.evaluate(x_train, y_train)
        # print('Accuracy on training set:')
        # print(f"precision, recall, f1: {[precision, recall, f1]}")

        precision, recall, f1 = model.evaluate(x_test, y_test)
        print('Accuracy on testing set:')
        print(f"precision, recall, f1: {[precision, recall, f1]}")
        r_res.extend([r, precision, recall, f1])
    res.append(r_res)

******* time:0 ********
-----train ratio:400-----
Accuracy on testing set:
precision, recall, f1: [0.75, 1.0, 0.8571]


In [29]:
import numpy as np
import statistics

p = np.array([x[1] for x in res])
r = np.array([x[2] for x in res])
f = np.array([x[3] for x in res])
print(statistics.mean(p), statistics.mean(r), statistics.mean(f))
print(max(abs(p - statistics.mean(p))))
print(max(abs(r - statistics.mean(r))))
print(max(abs(f - statistics.mean(f))))

0.84918 1.0 0.918354
0.04578000000000004
0.0
0.02735399999999999


In [ ]:
from multiprocessing import Pool
import tqdm
import time


def _foo(my_number):
    square = my_number * my_number
    time.sleep(1)
    return square


with Pool(2) as p:
    r = list(tqdm.tqdm(p.imap(_foo, range(30)), total=30))

In [29]:
import pandas as pd

res_pd = pd.DataFrame(res)
res_pd.to_csv("random_res_spirit.csv")

In [11]:
from ADR import preprocess
from ADR import sADR

train_ratio = [0.005, 0.01, 0.02]

for log_name, x_y_xColumns in log_datasets.items():
    print(f'=' * 30)
    print(log_name)
    x, y, xColumns = x_y_xColumns['x'], x_y_xColumns['y'], x_y_xColumns['xColumns']
    # break
    for r in train_ratio:
        # train_number = train_numbers[i]
        print(f'-----train ratio:{r}-----')
        # if i == 0:
        #     x_train, y_train, x_test, y_test = x_train, y_train, x_test, y_test = preprocess.split_to_train_test_by_num(x, y, num_train=train_number)
        # else:
        x_train, y_train, x_test, y_test = preprocess.split_to_train_test_by_ratio(x, y, train_ratio=r)
        # x_train = np.concatenate((x_train, x_train_adding), axis=0)
        # y_train = np.concatenate((y_train, y_train_adding), axis=0)

        # print(np.arange(x_train.shape[0]))
        model = sADR.sADR()
        model.fit(x_train, y_train)
        precision, recall, f1 = model.evaluate(x_train, y_train)
        print('Accuracy on training set:')
        print(f"precision, recall, f1: {[precision, recall, f1]}")

        precision, recall, f1 = model.evaluate(x_test, y_test)
        print('Accuracy on testing set:')
        print(f"precision, recall, f1: {[precision, recall, f1]}")

bgl
-----train ratio:0.005-----
Accuracy on training set:
precision, recall, f1: [0.9226, 1.0, 0.9598]
Accuracy on testing set:
precision, recall, f1: [0.8441, 1.0, 0.9155]
-----train ratio:0.01-----
Accuracy on training set:
precision, recall, f1: [0.9715, 1.0, 0.9856]
Accuracy on testing set:
precision, recall, f1: [0.9283, 1.0, 0.9628]
-----train ratio:0.02-----
Accuracy on training set:
precision, recall, f1: [0.9718, 1.0, 0.9857]
Accuracy on testing set:
precision, recall, f1: [0.9491, 1.0, 0.9739]


In [9]:
len(x)

69252

## uADR (unsupervised, do not need labelled logs for training)

In [4]:
from ADR import preprocess

u_log_datasets_train_test = {}

u_train_ratios = {
    'bgl': 0.8
}
for name, x_y_xColumns in log_datasets.items():
    if name in ['hdfs', 'bgl']:
        print("========")
        print(name)
        x, y, xColumns = x_y_xColumns['x'], x_y_xColumns['y'], x_y_xColumns['xColumns']
        print(y.sum() / y.size)
        print(f'x shape: {x.shape}')
        x_train, y_train, x_test, y_test = preprocess.split_to_train_test_by_ratio(x, y,
                                                                                   train_ratio=u_train_ratios[name])
        u_log_datasets_train_test[name] = [x_train, y_train, x_test, y_test]
        print(f'x_train shape:{x_train.shape}')
        print(f'x_test shape:{x_test.shape}')

bgl
0.4530555074221683
x shape: (69252, 384)
x_train shape:(55401, 384)
x_test shape:(13851, 384)


In [14]:
sum(y)

115

In [16]:
from ADR import uADR
from ADR import preprocess

log_name = 'bgl'
estimated_pN = 0.5

print('=' * 30)
print(log_name)
res = []
r_res = []
for estimated_pN in [0.9, 0.8]:
    for t in range(10):
        print(f'******* time:{t} ********')
        # print(f'estimated_pN: {estimated_pN}')
        x_train, y_train, x_test, y_test = preprocess.split_to_train_test_by_ratio(x, y, train_ratio=0.8)

        model = uADR.uADR(AN_ratio=1 - estimated_pN, nrows_per_sample=10, nrounds=100)
        print("training....")
        model.fit(x_train)
        # precision, recall, f1 = model.evaluate(x_train, y_train)
        # print('Accuracy on training set:')
        # print(f"precision, recall, f1: {[precision, recall, f1]}")
        print("testing....")
        precision, recall, f1 = model.evaluate(x_test, y_test)
        print('Accuracy on testing set:')
        print(f"precision, recall, f1: {[precision, recall, f1]}")
        r_res.extend([precision, recall, f])
    res.append(r_res)

testing....
Accuracy on testing set:
precision, recall, f1: [0.2788, 1.0, 0.4361]
******* time:0 ********
training....
testing....
Accuracy on testing set:
precision, recall, f1: [0.2212, 1.0, 0.3622]
******* time:0 ********
training....
testing....
Accuracy on testing set:
precision, recall, f1: [0.2019, 1.0, 0.336]
bgl
******* time:0 ********
training....
testing....
Accuracy on testing set:
precision, recall, f1: [0.2404, 1.0, 0.3876]
******* time:1 ********
training....
testing....
Accuracy on testing set:
precision, recall, f1: [0.2788, 1.0, 0.4361]
******* time:2 ********
training....
testing....
Accuracy on testing set:
precision, recall, f1: [0.25, 1.0, 0.4]
******* time:3 ********
training....
testing....
Accuracy on testing set:
precision, recall, f1: [0.2212, 1.0, 0.3622]
******* time:4 ********
training....
testing....
Accuracy on testing set:
precision, recall, f1: [0.2115, 1.0, 0.3492]
******* time:5 ********
training....
testing....
Accuracy on testing set:
precision, re

## References

[1] P. He, J. Zhu, Z. Zheng, and M. R. Lyu, “Drain: An Online Log Parsing Approach with Fixed Depth Tree,” in 2017 IEEE International Conference on Web Services (ICWS), Jun. 2017, pp. 33–40, doi: 10.1109/ICWS.2017.13.